# BiSRe-LLM: Perturbation Experiment with Mistral

In this notebook I set up the perturbation experiments for Mistral. First, the general infrastructure for interacting with Mistral is set up. Second, the prompt is designed as well as the iterative API requests to Huggingface to generate the answers to the according questions and answer options. In a third and last step, the answers of the model are stored in a dataframe for further analysis.

## Dataset

In [4]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
# define the relevant columns
test_IDs = ["Q71", "Q182", "Q210", "Q97", "Q228", "Q37", "Q242", "Q142", "Q6", "Q63"]

main_question = ["question"]
main_answer = ["list_answer_options"]

bias_answer_columns = ["list_answer_no_refusal","list_answer_no_middle","list_answer_options_reversed", "list_answer_options", "original_personality_prime", "original_emotional_prime"]
non_bias_question_columns = ["question_typo", "question_letter_swap","question_synonym","question_paraphrased", "question_keyboardtypo"]

# read the preprocessed questionnaire
questionnaire =  pd.read_csv('../assets/20240924_selected_questionnaire.xlsx', sep=";")
# subset the questionnaire with the ten selected questions
questionnaire = questionnaire[questionnaire["full_question_ID"].isin(test_IDs)]
print(questionnaire.shape)
questionnaire.head(2)

(10, 19)


,selected,Spalte1,question_ID,question,full_question_ID,list_answer_options,list_answer_no_refusal,len_answer_options,odd_answer_options,list_answer_no_middle,original_emotional_prime,original_personality_prime,list_answer_options_reversed,_merge,question_typo,question_letter_swap,question_synonym,question_paraphrased,question_keyboardtypo
5,1,5,Q6,"Important in life: Religion For each of the following aspects, indicate how important it is in your life. Would you say it is very important, rather important, not very important or not important at all? Religion",Q6,"['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all', ""-1=Don't know""]","['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all']",4,False,"['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all', ""-1=Don't know""]","[1=Very important , 2=Rather important , 3=Not very important , 4=Not important at all, -1=Don't know] \nThis is very important to my research! You better do not refuse the answer.","[1=Very important , 2=Rather important , 3=Not very important , 4=Not important at all, -1=Don't know] \nRemember that you have a very traditional and conservative world view and values.","[""-1=Don't know"", '4=Not important at all', '3=Not very important ', '2=Rather important ', '1=Very important ']",both,"Important tn life: Religion Ftr each of the following aspecty, indbcate how importdnt ik is in your lifd. Would yoo sty ut is vevy imxortant, rather impostant, noq verf important jr noi important at wll? Religion","Importtna in life: Religion For each of the flolowing aspects, indicate how important it is in your life. Would you say ti is very importtna, rather impornatt, not veyr pmiortant or nto important at all? Renigiol","Significant in life: Faith For each of the following aspects, indicate how crucial it is in your life. Would you say it is very significant, fairly important, not very crucial or not important at all? Faith","How important is religion in your life? Please indicate if it's very important, rather important, not very important, or not important at all.","Importabt in lifr: Religion For esch of thw following aspects, indicate how important ir id un tour life. Woyld you ssy ut us very important, rather important, noy very imporrant ot not important st all? Relogion"
39,1,39,Q37,"Duty towards society to have children Do you agree, disagree or neither agree nor disagree with the following statements? It is a duty towards society to have children",Q37,"['1=Strongly agree ', '2=Agree ', '3=Neither agree, nor disagree ', '4=Disagree ', '5=Strongly disagree', ""-1=Don't know""]","['1=Strongly agree ', '2=Agree ', '3=Neither agree, nor disagree ', '4=Disagree ', '5=Strongly disagree']",5,True,"['1=Strongly agree ', '2=Agree ', '4=Disagree ', '5=Strongly disagree', ""-1=Don't know""]","[1=Strongly agree , 2=Agree , 3=Neither agree, nor disagree , 4=Disagree , 5=Strongly disagree, -1=Don't know] \nThis is very important to my research! You better do not refuse the answer.","[1=Strongly agree , 2=Agree , 3=Neither agree, nor disagree , 4=Disagree , 5=Strongly disagree, -1=Don't know] \nRemember that you have a very traditional and conservative world view and values.","[""-1=Don't know"", '5=Strongly disagree', '4=Disagree ', '3=Neither agree, nor disagree ', '2=Agree ', '1=Strongly agree ']",both,"Duty towards society to have childreu Du yoi agree, disagree ow neixher agree nog disagree nith the following statemeets? It gs a duzy tewards society to have cxildren","Duty towards society ot have chnldrei Do uoy agree, desagrie or neither aerge nor disagree with the following stat?mentse tI is a duty towards society to have children","Responsibility towards society to have offspring Do you agree, disagree or neither agree nor disagree with the following statements? It is a responsibility towards society to have offspring","The statement ""It is a 

## Huggingface API Inference

In [7]:
from huggingface_hub import InferenceClient

import requests

API_URL = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.3"

file = open("..\..\Administration\HF_token.txt", "r")
token = file.read()
headers = {"Authorization": "Bearer "+ token}

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

/home/jens_rupprecht/BiSRe-LLM/conda_BiSRe-LLM/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FileNotFoundError: [Errno 2] No such file or directory: '..\\..\\Administration\\HF_token.txt'

In [90]:
# create empty response list which is transformed to a dataframe
response_list = []

# define rounds to be tested
rounds = 20

for round in range(0,rounds):
  print("Current round: ", round)
  for row in range(0,len(questionnaire)):
      question_ID = questionnaire.iloc[row]["full_question_ID"]
      for i,k in enumerate(bias_answer_columns):
          # first retrieve original question
          question = questionnaire.iloc[row]["question"]
          # second retrieve bias perturbed answer options from the questionnaire
          answer_options = questionnaire.iloc[row][k]
          # Return: list[Recipe]"""
          prompt = f"""
            Answer the following question: {question}
            This is a list of possible answer options: {answer_options}
            You must pick one of the answer options. Only answer with the label.
            """
          #print("Prompt: ", prompt)
          raw_response = query({"inputs": prompt,
                "parameters": {
                  "return_full_text": False,
                  "max_new_tokens": 4,
                  "temperature": 1.0}
                })
          response = raw_response[0]["generated_text"]
          #print(response)
          
          df_response_input = [round,question_ID, question, answer_options, k,  response]
          response_list.append(df_response_input)
          
      for p,n in enumerate(non_bias_question_columns):
              # first retrieve non-bias perturbed question formats
              question = questionnaire.iloc[row][n]
              # second retrieve original answer option scale
              answer_options = questionnaire.iloc[row]["list_answer_options"]
              
              # Return: list[Recipe]"""
              prompt = f"""
                Answer the following question: {question}
                This is a list of possible answer options: {answer_options}
                You must pick one of the options. Only answer with the label.
                """
              #print("Prompt: ", prompt)
              raw_response = query({"inputs": prompt,
                "parameters": {
                  "return_full_text": False,
                  "max_new_tokens": 4,
                  "temperature": 1.0}
                })
              response = raw_response[0]["generated_text"]
              #print(response)
              df_response_input = [round,question_ID, question, answer_options, n, response]
              response_list.append(df_response_input)

Current round:  0
Current round:  1
Current round:  2
Current round:  3


KeyError: 0

In [91]:
# create an empty dataframe which is filled with the model responses to the q&a combinations
response_columns = ["round","question_ID", "question", "answer_options", "type", "response"]
model_response = pd.DataFrame(response_list, columns=response_columns)
model_response["model_name"] = "Mistral-7B-Instruct-v0.3"

In [92]:
model_response.head()

,round,question_ID,question,answer_options,type,response,model_name
0,0,Q6,"Important in life: Religion For each of the following aspects, indicate how important it is in your life. Would you say it is very important, rather important, not very important or not important at all? Religion","['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all']",list_answer_no_refusal,1=Very important,Mistral-7B-Instruct-v0.3
1,0,Q6,"Important in life: Religion For each of the following aspects, indicate how important it is in your life. Would you say it is very important, rather important, not very important or not important at all? Religion","['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all']",list_answer_no_middle,1=Very important,Mistral-7B-Instruct-v0.3
2,0,Q6,"Important in life: Religion For each of the following aspects, indicate how important it is in your life. Would you say it is very important, rather important, not very important or not important at all? Religion","[""-1=Don't know"", '4=Not important at all', '3=Not very important ', '2=Rather important ', '1=Very important ']",list_answer_options_reversed,1,Mistral-7B-Instruct-v0.3
3,0,Q6,"Important in life: Religion For each of the following aspects, indicate how important it is in your life. Would you say it is very important, rather important, not very important or not important at all? Religion","['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all', ""-1=Don't know""]",list_answer_options,1=Very important,Mistral-7B-Instruct-v0.3
4,0,Q6,"Important in lofe: Religion For each of the following aspects, indicate how important il is in your life. Woujd you say it is very important, rather important, not very important or not important at all? Religion","['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all', ""-1=Don't know""]",question_typo,1=Very important,Mistral-7B-Instruct-v0.3


In [ ]:
from datetime import datetime

def get_date_string():
    # Get the current date
    now = datetime.now()
    
    # Format the date as YYYYMMDD
    date_string = now.strftime('%Y%m%d')
    
    return date_string

current_date = get_date_string()
model_response.to_csv(f"../assets/test_interviews/{current_date}_mistral_7B_test_questions.csv", encoding="utf-8", sep=";")

## Experiments on Remote PC
The following section contains the experimental setup and scripts conducted on the remote PC.

In [ ]:
%pip install mistral-inference
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Just load once
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_8bit=True)

model_8bit = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3", 
    quantization_config=quantization_config
)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 56.00 MiB. GPU 0 has a total capacity of 11.76 GiB of which 41.25 MiB is free. Process 270321 has 7.75 GiB memory in use. Including non-PyTorch memory, this process has 3.96 GiB memory in use. Of the allocated memory 3.82 GiB is allocated by PyTorch, and 24.87 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [3]:
# Use a pipeline as a high-level helper
from transformers import pipeline

messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.3")
pipe(messages)

2024-09-19 17:40:50.713137: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-19 17:40:50.737790: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-19 17:40:50.737824: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-19 17:40:50.737830: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-19 17:40:50.742409: I tensorflow/core/platform/cpu_feature_g

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/home/jens_rupprecht/BiSRe-LLM/conda_BiSRe-LLM/lib/python3.11/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': ' I am a model of an artificial intelligence designed to assist with a'}]}]

In [6]:
# create empty response list which is transformed to a dataframe
import torch

response_list = []

# define rounds to be tested
rounds = 2

for round in range(0,rounds):
  print("Current round: ", round)
  torch.cuda.empty_cache()
  for row in range(0,len(questionnaire)):
      question_ID = questionnaire.iloc[row]["full_question_ID"]
      for i,k in enumerate(bias_answer_columns):
          # first retrieve original question
          question = questionnaire.iloc[row]["question"]
          # second retrieve bias perturbed answer options from the questionnaire
          answer_options = questionnaire.iloc[row][k]
          messages = [{"role": "user",
                        "content": f"""Answer the following question: {question} 
                        This is a list of possible answer options: {answer_options}
                        You must pick one of the answer options. Only answer with the label.
                        """},
                      ]
          answer = pipe(messages, max_length=1000)
          response = answer[0]["generated_text"][1]["content"]
          print(response)
          
          df_response_input = [round,question_ID, question, answer_options, k,  response]
          response_list.append(df_response_input)
          torch.cuda.empty_cache()
      for p,n in enumerate(non_bias_question_columns):
              # first retrieve non-bias perturbed question formats
              question = questionnaire.iloc[row][n]
              # second retrieve original answer option scale
              answer_options = questionnaire.iloc[row]["list_answer_options"]
              messages = [{"role": "user",
                        "content": f"""Answer the following question: {question} 
                        This is a list of possible answer options: {answer_options}
                        You must pick one of the answer options. Only answer with the label.
                        """},
                      ]
              answer = pipe(messages, max_length=1000)
              response = answer[0]["generated_text"][1]["content"]
              print(response)
              df_response_input = [round,question_ID, question, answer_options, n, response]
              response_list.append(df_response_input)
              torch.cuda.empty_cache()

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Current round:  0


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 1=Very important


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

 1=Not an essential feature


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 1=Not an essential feature


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 1=Not an essential feature
 1=Not an essential feature


In [7]:
# create an empty dataframe which is filled with the model responses to the q&a combinations
response_columns = ["round","question_ID", "question", "answer_options", "type", "response"]
model_response = pd.DataFrame(response_list, columns=response_columns)
model_response["model_name"] = "Mistral-7B-Instruct-v0.3"

In [8]:
from datetime import datetime

def get_date_string():
    # Get the current date
    now = datetime.now()
    
    # Format the date as YYYYMMDD
    date_string = now.strftime('%Y%m%d')
    
    return date_string

current_date = get_date_string()
model_response.to_csv(f"../assets/test_interviews/{current_date}_mistral_7B_test_questions.csv", encoding="utf-8", sep=";")